In [1]:
import torch
from torch.utils.data.dataloader import DataLoader

import sys

# Setting path so as we can find files in ../src folder
sys.path.append('../src')

from BRATS2013Dataset import BRATS2013Dataset

from tqdm.autonotebook import tqdm

import numpy as np

import json

import skimage

from pprint import pprint

from matplotlib import pyplot as plt

import modin.pandas as pd

import os
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Dask




In [2]:
DATASET_NAME = "brats_2013"
DATASET_SPLIT = "test"
DATASET_PATH = f"../data/{DATASET_NAME}_obs_path_list_unstacked_resized_cleaned_{DATASET_SPLIT}.txt"

obs_list = []

with open(DATASET_PATH) as file:
  for line in file:
    obs_list.append(line.rstrip())

In [3]:
PATCH_SIZE = 33

In [4]:
dataset = BRATS2013Dataset(
  obs_list=obs_list, stage=DATASET_SPLIT, patch_size=PATCH_SIZE
)

In [5]:
df_list = []

for entry in tqdm(dataset):
  patch_metadata = entry["patch_metadata"]
  img_path = entry["full_path"]

  for label in patch_metadata:
    for patch_id in patch_metadata[label]:
      
      df_list.append(
        {
          "patch_id": patch_id,
          "label": label,
          "img_path": img_path
        }
      )



  0%|          | 0/331 [00:00<?, ?it/s]

In [6]:
df = pd.DataFrame(df_list)

2022-12-30 11:27:50,799	INFO worker.py:1538 -- Started a local Ray instance.


In [7]:
PATCH_DATASET_PATH = f"../data/{DATASET_NAME}_patch_{PATCH_SIZE}_df_{DATASET_SPLIT}.json"

In [8]:
df.to_json(PATCH_DATASET_PATH)
